In [53]:
import pandas as pd
import numpy as np
import glob
from nested_dict import nested_dict

In [54]:
df_ans = nested_dict()

In [55]:
# Testnet / Exp 1 / (avg, std, gas)
path = "../experiments/*.csv"
for fname in glob.glob(path):
    # Parse file name
    second_slash_pos = fname.index("/",3)
    underscore_pos = fname.index("_")
    hash_pos = fname.index("#")
    
    # Locate Testnet and Experiment Number
    testnet = fname[second_slash_pos+1:underscore_pos]
    expNum = fname[hash_pos+1:hash_pos+3].rstrip()
    print(testnet, expNum)

    # Parse data
    df = pd.read_csv(fname, names=range(10))
    gas = df[5][1] # conventional gas usage
    avg = df.mean(axis=1)[0]
    std = df.std(axis=1)[0]
    print(avg, std, gas)
    
    # Collect data
    df_ans[testnet][expNum]["avg"] = avg
    df_ans[testnet][expNum]["std"] = std
    df_ans[testnet][expNum]["gas"] = gas


Kovan 3
9644.4 6341.59599961 1085612
Rinkeby 1
30077.1 2787.47354546 917439
Rinkeby 6
30020.9 294.192095218 73524
Ropsten 7
28619.1 10963.4633665 118304
Rinkeby 9
30002.0 472.844348362 476544
Ropsten 8
35434.6 18609.1997153 207864
Ropsten 10
49414.0 22919.2577784 924344
Kovan 6
7633.4 4774.72154395 73524
Rinkeby 8
30082.6 587.080384426 207864
Ropsten 9
46657.6 23112.5961146 476544
Kovan 2
8452.5 4359.21736732 234804
Kovan 4
9500.8 4690.34430861 2149122
Ropsten 6
31211.9 13891.3257223 73524
Rinkeby 7
29938.7 281.668856007 118304
Kovan 10
10036.9 4679.41151215 924344
Ropsten 2
51581.9 23289.8511395 234804
Ropsten 1
40560.2 16687.6210461 917439
Ropsten 5
68015.7 37514.5402479 37700
Kovan 8
10784.1 5510.2397901 207864
Kovan 5
10893.3 6650.84205279 37700
Rinkeby 3
30050.0 604.805753941 1085612
Kovan 1
12897.1 4854.45006497 917439
Kovan 7
10394.6 5439.27714152 118304
Rinkeby 2
29545.9 1372.28766421 234804
Ropsten 4
56884.1 42016.5504625 2149122
Rinkeby 10
30041.5 723.376534808 924344
Rinkeby

In [58]:
df_ans['Ropsten']['5']['gas']

37700